#### AED Location Optimization

In this notebook, we build a class `AEDLocationOptimizer` to optimize the location of AED. There are 2 types optimization embeded in the class. 1. Maximal Coverage Location Problem (`mclp`) and 2. Location Set Coverage Problem (`lscp`). The first algorithm tried to cover the maximal number of interventions given a certain number of AEDs. The second one tried to find the minimal number of AEDs that can cover all interventions in the dataset. Since the Kmeans already gave a almost perfect coverage improvement (99.6%) with the given number of AEDs. We only apply the `lscp` algorithm on interventions in Brussels. The result show that we only need 598 AEDs in Brussels to cover all AED related interventions, comparing to 925 AEDs in Brussels after Kmeans optimization.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from scipy.stats import gaussian_kde
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from shapely.geometry import Polygon, Point
from gurobipy import Model, GRB, quicksum
from pyproj import Geod

In [2]:
itv_aed = pd.read_csv(
    '/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/intervention_aed_kmeans_distance.csv',
    low_memory=False)

aed = pd.read_csv(
    '/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/aed_with_KmeansLocation.csv')

## Drop duplicates
itv_aed.drop_duplicates(subset=['mission_id'], keep='first', inplace=True)
itv_aed.drop_duplicates(subset=['lat_itv', 'lon_itv'], keep='first', inplace=True)

itv_aed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25381 entries, 0 to 25380
Data columns (total 57 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   mission_id                        25381 non-null  int64  
 1   service_name                      22837 non-null  object 
 2   postalcode_permanence             16087 non-null  float64
 3   cityname_permanence               16830 non-null  object 
 4   streetname_permanence             16878 non-null  object 
 5   housenumber_permanence            1366 non-null   float64
 6   latitude_permanence               23546 non-null  float64
 7   longitude_permanence              23741 non-null  float64
 8   permanence_short_name             25350 non-null  object 
 9   permanence_long_name              22839 non-null  object 
 10  vector_type                       25049 non-null  object 
 11  eventtype_firstcall               16135 non-null  object 
 12  even

In [7]:
class AEDLocationOptimizer: # AED location optimization class

    def __init__(
            self,
            df,  # dataframe: AED related intervention data
            num_aed,  # number of potential AED locations
            optim_type='mclp',  # 'mclp': maximal covering location problem / 'lscp': location set covering problem
            golden_minutes=4,  # minutes, golden_minutes: best interval for cpr using AED
            speed_running=100):  # m/min, 6 km/hour

        self.df = df
        self.num_itv = self.df.shape[0]
        self.num_aed = num_aed
        self.optim_type = optim_type
        self.golden_minutes = golden_minutes
        self.speed_running = speed_running
        self.r = self.speed_running * golden_minutes / 2
        self.geod = Geod(ellps='WGS84') # Geodesic distance calculator
        self.hull = ConvexHull(self.df[['lon_itv', 'lat_itv']])

    def generate_aed_locations(self, method='kmeans'):
        if method == 'random': # Uniformly generate AED locations within the convex hull
            min_lon, min_lat = np.min(self.df['lon_itv']), np.min(self.df['lat_itv'])
            max_lon, max_lat = np.max(self.df['lon_itv']), np.max(self.df['lat_itv']) # Bounding box of intervention locations

            aed = []
            while len(aed) < self.num_aed: # Generate AED locations until reaching the desired number
                lon = np.random.uniform(min_lon, max_lon)
                lat = np.random.uniform(min_lat, max_lat)
                if Polygon(self.hull.points[self.hull.vertices]).contains(Point(lon, lat)): # Check if the AED location is within the convex hull
                    aed.append([lon, lat])
            return np.array(aed)

        elif method == 'kde': # Generate AED locations based on the density of intervention locations (kernel density estimation) within the convex hull
            kernel = gaussian_kde(self.df[['lon_itv', 'lat_itv']].values.T,
                                  bw_method='scott') # 'silverman': Silverman's rule of thumb has wider bandwidth than default Scott's rule of thumb

            min_lon, min_lat = np.min(self.df['lon_itv']), np.min(self.df['lat_itv'])
            max_lon, max_lat = np.max(self.df['lon_itv']), np.max(self.df['lat_itv'])
            aed = []
            while len(aed) < self.num_aed:  # Generate AED locations until reaching the desired number
                lon = np.random.uniform(min_lon, max_lon)
                lat = np.random.uniform(min_lat, max_lat)
                if Polygon(self.hull.points[self.hull.vertices]).contains(Point(lon, lat) # Check if the AED location is within the convex hull
                                            ) and np.random.random() < kernel([lon, lat])[0]: # Check if the prob/density of this location is greater than ramdom(0, 1)
                    aed.append((lon, lat))
            return np.array(aed)

        elif method == 'kmeans': # Generate AED locations based on clustering of intervention locations
            kmeans = KMeans(n_clusters=self.num_aed, random_state=0).fit(self.df[['lon_itv', 'lat_itv']]) # Fit KMeans model
            aed = kmeans.cluster_centers_
            return aed

        elif method == 'gmm': # Generate AED locations based on Gaussian Mixture Model
            gmm = GaussianMixture(n_components=self.num_aed, random_state=0).fit(itv_aed[['lon_itv', 'lat_itv']]) # Fit GMM model
            aed = gmm.means_
            return aed

        else:
            raise ValueError('Invalid method')

    def calculate_distance(self, lon1, lat1, lon2, lat2): # Calculate geodesic distance between two points
        _, _, distance = self.geod.inv(lon1, lat1, lon2, lat2)
        return np.float32(distance)

    def get_distance_matrix(self, aed_location): # Calculate the distance matrix between intervention locations and AED locations
        distance_matrix = [[
            self.calculate_distance(self.df.iloc[i]['lon_itv'],
                                    self.df.iloc[i]['lat_itv'], location[0],
                                    location[1]) for location in aed_location
        ] for i in range(self.num_itv)]
        return np.array(distance_matrix)

    def optimize(self, aed_location, num_devices=0):  # num_devices: number of AEDs to be placed when using 'mclp'
        distance_matrix = self.get_distance_matrix(aed_location)
        model = Model(self.optim_type)
        if self.optim_type == 'mclp':
            x = model.addVars(self.num_aed, vtype=GRB.BINARY, name="x") # Decision variable: 1 if AED is placed, 0 otherwise
            y = model.addVars(self.num_itv, vtype=GRB.BINARY, name="y") # Decision variable: 1 if intervention is covered, 0 otherwise

            model.addConstr(
                quicksum(x[j] for j in range(self.num_aed)) == num_devices, "num_devices") # Constraint: number of AEDs to be placed

            for i in range(self.num_itv):
                model.addConstr(
                        quicksum(x[j] for j in range(self.num_aed) if distance_matrix[i][j] <= self.r)
                                >= y[i], f"cover_{i}")  # Constraint: each intervention is covered by at least one AED

            model.setObjective(
                quicksum(y[i] * self.df.iloc[i]['hospital_distance'] for i in range(self.num_itv)),
                GRB.MAXIMIZE) # Objective function: maximize the utility of AED placement, using the distance to the nearest hospital as the weight

        elif self.optim_type == 'lscp':
            ## Set hyperparameters to speed up the optimization process
            #model.setParam('MIPFocus', 1)  # focus on finding first feasible solutions
            #model.setParam('Heuristics', 0.01)  # 1% heuristics to stop exploring suboptimal solutions

            x = model.addVars(self.num_aed, vtype=GRB.BINARY, name="x")  # Decision variable: 1 if AED is placed, 0 otherwise

            for i in range(self.num_itv):
                model.addConstr(
                    quicksum(x[j] for j in range(self.num_aed) if distance_matrix[i][j] <= self.r) >= 1,
                    f"cover_{i}") # Constraint: each intervention is covered by at least one AED

            model.setObjective(
                quicksum(x[j] for j in range(self.num_aed)), GRB.MINIMIZE)  # Objective function: minimize the number of AEDs

        else:
            raise ValueError(
                "Unsupported optimization type: choose 'mclp' or 'lscp'")

        model.optimize()  # Solve the model

        if model.status == GRB.OPTIMAL:
            optim_location = []
            print('Max Utility:', model.objVal) if self.optim_type == 'mclp' else print('Min AEDs:', model.objVal)
            for i in range(self.num_aed):
                if x[i].X > 0.5:  # consider the AED location if x[i] > 0.5 (i.e., x[i] == 1) since the calculated value is not exactly 1
                    optim_location.append(aed_location[i])

            df_optim_location = pd.DataFrame(optim_location,
                                          columns=['lon', 'lat'])
            return df_optim_location

        else:
            print('No solution found')


In [4]:
def find_closest_distance(df1, df2): # Find the closest distance between two sets of points
    geod = Geod(ellps='WGS84')
    distances = []
    for index1, row1 in df1.iterrows():
        min_distance = float('inf')
        for index2, row2 in df2.iterrows():
            # calculate the distance between two points
            _, _, distance = geod.inv(row1['lon_itv'], row1['lat_itv'],
                                      row2['lon'], row2['lat'])
            # update the min distance
            if distance < min_distance:
                min_distance = distance
        # append the min distance to the list
        distances.append(min_distance)
    return distances

def get_coverage(df, aed_location, r=200): # Calculate the coverage of AED placement
    df = df.copy()
    df['nearest_distance'] = find_closest_distance(df, aed_location)
    covered = df.loc[(
        (df['nearest_distance'] <= r) | # Check if the distance to the nearest AED is less than r
        (df['hospital_distance'] <= 2000))] # Check if the distance to the nearest hospital is less than 2000m # driving speed 500m/min for 4 minutes
    
    coverage = len(covered) / len(df)
    return coverage


In [5]:
## Add province information to the optimized AED locations

import geopandas as gpd
from shapely import geometry as geo
from shapely.validation import explain_validity

geo_path = '/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/Belgium.provinces.WGS84.geojson'
geo_be = gpd.read_file(geo_path)

# Check if the geometries are valid
for i in range(len(geo_be)):
    if not geo_be.loc[i, 'geometry'].is_valid:
        print(explain_validity(geo_be.loc[i, 'geometry']))
        geo_be.loc[i, 'geometry'] = geo_be.loc[i, 'geometry'].buffer(0)
        print(geo_be.loc[i, 'geometry'].is_valid)


def get_medical_province(df, geo_df):
    province = []
    missing_province = 0
    for i in range(len(df)):
        point = geo.Point(df.loc[i, 'lon'], df.loc[i, 'lat'])
        contained = geo_df.loc[geo_df['geometry'].contains(
            point)]['NameDUT'].values
        if contained.size > 0:
            province.append(contained[0])
        else:
            province.append(None)
            missing_province += 1

    df['new_province'] = province
    print(f'{missing_province} coordinates are not located in any province')

    return df

Self-intersection[6.24760990547934 50.640636186645]
True


In [6]:
## LSCP optimization for AED placement in Brussels

df_brl = itv_aed.loc[itv_aed['province']=='Brussels Hoofdstedelijk Gewest']
df_brl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2820 entries, 0 to 19649
Data columns (total 57 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   mission_id                        2820 non-null   int64  
 1   service_name                      2792 non-null   object 
 2   postalcode_permanence             2078 non-null   float64
 3   cityname_permanence               2789 non-null   object 
 4   streetname_permanence             2789 non-null   object 
 5   housenumber_permanence            624 non-null    float64
 6   latitude_permanence               2506 non-null   float64
 7   longitude_permanence              2506 non-null   float64
 8   permanence_short_name             2792 non-null   object 
 9   permanence_long_name              2792 non-null   object 
 10  vector_type                       2506 non-null   object 
 11  eventtype_firstcall               2078 non-null   object 
 12  eventlevel

In [8]:
optimize_brl = AEDLocationOptimizer(df_brl, 20000, 'lscp')
aed_location_brl = optimize_brl.generate_aed_locations(method='kde')
optim_location_brl = optimize_brl.optimize(aed_location_brl)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-05
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2820 rows, 20000 columns and 46023 nonzeros
Model fingerprint: 0x832e279b
Variable types: 0 continuous, 20000 integer (20000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 855.0000000
Presolve removed 1972 rows and 18992 columns
Presolve time: 0.06s
Presolved: 848 rows, 1008 columns, 4237 nonzeros
Found heuristic solution: objective 641.0000000
Variable types: 0 continuous, 1008 integer (1008 binary)

Root relaxation: objective 5.935511e+02, 1368 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objec

In [14]:
coverage = get_coverage(df_brl, optim_location_brl)
print('Coverage rate in Brussels after optimization:', coverage,
      '\nThe minimal number of AEDs:', len(optim_location_brl))

Coverage rate in Brussels after optimization: 1.0 
The minimal number of AEDs: 598


In [15]:
optim_location_brl.to_csv('/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/aed_brl_lscp.csv', index=False)